## Part 1: Introduction to Non Deep Learning Methods

Before we get into deep learning, let's talk about non deep learning methods so  

### 1. Linear Classifier

#### 1.1 Classification vs Regression

Machine learning can be thought of as statistical dataset learning. That is, a system "learns" by building a statistical model from its "experience," which is provided to it in the form of a dataset. Machine learning has been used for a multitude of tasks in areas such as diagnosis, recognition, and planning. We can broady categorize all of these tasks as providing some kind of prediction: in the case of diagnosis, we are predicting something diagnostic, in the case of recognition, we are predicting in the form of recognizing an object or pattern, and in the case of planning, we are predicting the next optimal move in our plan. Broadly, this prediction can come in the form of classification and regression. The below image visually depicts the difference [8]:

<img src="classification_vs_regression.jpg" width="600" height="480" />


While classification predicts a binary label (between 0 and 1 or 1 and -1 depending on how the problem frames it), regression predicts a numerical value (any floating point number). The image above in particular shows a line or linear classifier on the left, which separates the positive (typically labeled 1) and negative labels (typically labelled either 0 or -1). The linear regressor on the right models a line that fits a set of numerical values.

The extension of classification above to multiple classes is simply extending the label set from {0, 1} to {0, 1, ... K} where K = the number of possible classes. This is known as multiclass classification.

We will focus on multiclass classification in this notebook, but note that we could easily reduce down to only two classes in order to structure it as _binary classification_.

#### 1.2 Motivating example

**NOTE: The rest of the notebook will be primarily derived from CS231N.**

A classic problem that requires multiclass classification is image classification, where you have to come up with a way to take an input image and come up with a single label (or a distribution as shown below)  that captures what is in the image, drawn from a set of predefined image labels [2]. 

<img src="classify.jpg" />


In the image above, "an image classification _model_ takes a single image and assigns probabilities to 4 labels, {cat, dog, hat, mug}. As shown in the image, keep in mind that to a computer an image is represented as one large 3-dimensional array of numbers."  For the purposes of this notebook, we will assume the image was drawn from the CIFAR-10 dataset [9] and was enlarged for clarity. The CIFAR-10 "dataset consists of 60,000 tiny images that are 32 pixels high and wide. Each image is labeled with one of 10 classes (for example 'airplane', 'automobile', 'bird', etc). These 60,000 images are partitioned into a training set of 50,000 images and a test set of 10,000 images." [2]

Let us assume the original cat image (the tiny one that was not enlarged) is 32 pixels wide, 32 pixels tall, and has three color channels Red, Green, Blue (or RGB). Therefore, the image consists of 32 x 32 x 3 numbers for a total of 3072 numbers. Each number represents a color channel of a pixel and its value ranges from 0 (black) to 255 (white). Thus, if we wanted to perform image classification, our task would be to turn the image, represented as 3072 numbers, into a single label, such as 'cat.' 

We will use the _training set_ of example images later on to _train_ a classification model and then evaluate its performance on a set of images the model has not encountered, the _test set_. Why does this work theoretically speaking? It is because we assume the examples are drawn from an independent and identically distributed set of images (this is known as the i.i.d assumption). Thus, the model should in theory be able to _generalize_ to a test dataset given a training dataset [4]. (Note: there is also a third kind of dataset, known as a validation dataset, that is typically seen in machine learning literature. The validation dataset is a subset of the training dataset that we use to help us pick various _hyperparameters_ of whatever model we pick. More on this later.)

#### 1.3 Linear Regression function

**Warning: Math below! **

Our initial approach to image classification will naturally extend to deep learning. "The approach will have two major components: a score function that maps the raw data to class scores, and a loss function that quantifies the agreement between the predicted scores and the ground truth labels. We will then cast this as an optimization problem in which we will minimize the loss function with respect to the parameters of the score function." [2]

Let us assume we have a matrix of input features $\boldsymbol{X}$, consisting of m (m = number of examples) rows, where each row $\boldsymbol{x}^{(i)}$ corresponds to a vector of n feature values $[\boldsymbol{x}^{(i)}_{1}, \boldsymbol{x}^{(i)}_{2}, ... \boldsymbol{x}^{(i)}_{n}]$.  Thus, $\boldsymbol{X}$ is a m x n matrix. We also have a vector of m labels $\boldsymbol{y}$, with $y_{i}$ corresponding to the _class_ or _label_ for example $i$. The score function will take the form of a linear regression function:

$\hat{y} = \boldsymbol{w}^{T}\boldsymbol{x} + b$


$\hat{y}$ is the value our classification model predicts $y$ should take on [4].

$\boldsymbol{x}$ corresponds to a row of input features from our matrix $\boldsymbol{X}$ above and $\boldsymbol{w}$ is a vector of n _weights_ that determine how each feature in $\boldsymbol{x}$ affects the prediction. A positive weight means increasing the value of that feature increase our predicted value $\hat{y}$, and a negative weight means increasing the value of that feature decreases our predicted value. And if a feature has a weight of zero, it means it does not have any impact on the prediction.

$b$ refers to a bias parameter, which is the value of the prediction if all of the weights were 0.

If we wanted to extend this to multi-class, we would write w as W since it would now be a matrix of weights, one row for each class:

$\hat{y} = \boldsymbol{W}^{T}\boldsymbol{x} + b$

We can simplify the equation to:

$\hat{y} = \boldsymbol{W}^{T}\boldsymbol{x}$

if we assume that $\boldsymbol{x}$ is augmented by an extra entry that is always set to 1 and the weight corresponding to this extra entry is the bias parameter.  This is known as the bias trick and is illustrated below [2]:

<img src="wb.jpg" width="600" height="480" / >

Our task is thus to learn a set of weights $\boldsymbol{w}$ that results in the best model performance. How do we measure model performance? There was a number of different measures, and one common one is to the compute the mean squared error (MSE) of the model on the test test.  The $MSE_{test}$ is given by [4]:

$MSE_{test}$ = $\frac{1}{m} \sum_{i=1} ( \hat{y}^{(test)} - {y}^{(test)} )_{i}^{2}$

where $\hat{y}^{(test)}$ is the predictions of the model on the test set and ${y}^{(test)}$ is the vector of actual regression _targets_ from the test set.  Thus, mean squared error is essentially a sum of a function of prediction - target over all the input exmaples.

Remember that while we evaluate the model performance on the test set, the model only gets to see the training set. Thus, we can only work to minimize $MSE_{train}$ and then see if it generalizes to the test set. The nice thing about linear regression is that you can actually solve a system of equations (that will not be described here) for a set of weights $\boldsymbol{w}$ that minimizes $MSE_{train}$.

### 2. Multiclass Support Vector Machine Classifier

Note that in the section above we initially started talking about classfication, which requires a model to be able to predict 0/1, and then we ended up introduction linear regression, which actually outputs a score (that is not necessarily between 0 and 1).  How do we reconcile this?

We can use the image classification use case to understand how to combine these two ideas together.

<img src="imagemap.jpg" />

The above is "an example of mapping an image to class scores. For the sake of visualization, we assume the image only has 4 pixels (4 monochrome pixels, we are not considering color channels in this example for brevity), and that we have 3 classes (red (cat), green (dog), blue (ship) class). (Clarification: in particular, the colors here simply indicate 3 classes and are not related to the RGB channels.) We stretch the image pixels into a column and perform matrix multiplication to get the scores for each class. Note that this particular set of weights W is not good at all: the weights assign our cat image a very low cat score. In particular, this set of weights seems convinced that it's looking at a dog." [2]

**The class (or label) with the highest score is the one the model predicts for the image** We are going to measure how well the model does with a _loss function_, also known as a cost function or objective function.  The loss should be low if we are doing a good job of classifying the training data and high if we are not doing a good job.

Below are notes that detail a common loss function [2]:

<img src="multiclass_svm_loss_1.jpg" />

<img src="multiclass_svm_loss_2.jpg" />


In the next cell, you'll implement the SVM loss function for a single example L_i in Python.

In [22]:
import numpy as np
def L_i(x, y, W):
  """
  Compute the cross-entropy loss for a single example (x,y)
  - x is a column vector representing an image (e.g. 3073 x 1 in CIFAR-10)
    with an appended bias dimension in the 3073-rd position (i.e. bias trick)
  - y is an integer giving index of correct class (e.g. between 0 and 9 in CIFAR-10)
  - W is the weight matrix (e.g. 10 x 3073 in CIFAR-10)
  """
  # TODO: implement using delta = 1.0
  # Solution at the end of the notebook

print "Done"

Done


### 3. Multinomial Logistic Regression aka Softmax Classifier

Here's a useful analogy to think about a few classifiers we discuss in this notebook:

Support Vector Machine Classifier :: Multiclass Support Vector Machine Classifier

as

Logistic Regression Classifier :: Multinomial Logistic Regression Classifier (also known as Softmax classifier)

That's the analogy to keep in mind. The Support Vector Machine Classifier and Logistic Regression Classifier are for the binary classification use case: the former outputs a class label whereas the latter outputs a probability of the positive class label.  The multiclass Support Vector Machine Classifier and multinomial Logistic Regression Classifier are for the multiclass classification use case: the former outputs a set of class scores whereas the latter outputs a probability distribution across all the labels. In the multiclass class, you can get the predicted label by finding the class with the highest score or highest probability respectively.

Below are notes on the loss function used to construct the softmax classifier [2]:

<img src="cross_entropy_loss.jpg" />


The Softmax classifier is minimizing the "cross-entropy" loss (fancy term for difference between two probability distributions) between the estimated class probabilities and the actual distribution, which we take to be a distribution where probability = 1 on the correct class and 0 elsewhere (i.e. p=[0,…1,…,0] contains a single 1 at the yi -th position corresponding to the correct label). 

We won't implement this function in Python but I leave it as an exercise to the reader!

### 4. What goes into a machine learning algorithm?

As Goodfellow et. al states, **nearly all machine learning algorithms can be characterized by the following recipe: "a specification of a dataset, a cost function, an optimization procedure and a model."**[6] We can mix and match these to produce different algorithms. In the case of deep learning, our model will be a neural network, our optimization procedure will be typically something called stochastic gradient descent, and our cost/loss function can come from anything in the family of classification loss functions. We already saw two classification loss functions above: hinge loss and cross-entropy loss. How do we find the best weights with respect to this loss function? Enter gradient descent, which is the main method used in most machine learning algorithms.


### 5. Gradient Descent


To get a nice overview, first checkout this video:
[Gradient Descent Video](https://youtu.be/5u0jaA3qAGk?t=33s)


As we learned above, the major goal for learning a machine learning model is to find the optimal set of weights that minimizes the loss function we define. One way to do this could be to randomly search the set of weights but the space is high dimensionsal and this search would be slow [2].

A better way to find the optimal set of weights is to use _gradient descent_, which is an optimization algorithm that lets us use the gradient of the loss function with respect to the weights.

In [18]:
# Modified from http://cs231n.github.io/optimization-1/

"""
Vanilla Gradient Descent
Our gradient descent algorithm is quite simply, for each training example we get, compute a weight update as follows:
weights += learning_rate * gradient of weights with respect to loss function
our gradient has the same dimensions as the weights vector (or matrix) itself
"""

# NOTE: this code sample won't work. Just meant to illustrate algorithm
while True:
  weights_gradient = evaluate_gradient(loss_function, data, weights)
  weights += - learning_rate * weights_gradient # perform parameter update

NameError: name 'evaluate_gradient' is not defined

The "vanilla" gradient descent is inefficient because a single update of the weights results in us having to process over all the training examples (captured by the variable data) above. Instead, we can do what's known as "mini-batch" gradient descent, which samples a small amount of training examples to perform each update.

In [19]:
# Borrowed from http://cs231n.github.io/optimization-1/

"""
Vanilla Minibatch Gradient Descent
"""

# NOTE: this code sample won't work. Just meant to illustrate algorithm
while True:
  # in current state of the art ConvNets, a typical batch contains 256 examples
  # this batch is then used to perform a parameter update:
  data_batch = sample_training_data(data, 256)
  weights_grad = evaluate_gradient(loss_fun, data_batch, weights)
  weights += - step_size * weights_grad # perform parameter update

NameError: name 'sample_training_data' is not defined

The below uses a good analogy of finding the path down a hill to illustrate gradient descent [12]:

<img src="gradient_descent_analogy.jpg" width="800" height="640" />

Let's us the example of Linear regression with Mean squared error cost function (since the gradient is easy to compute) to illustrate gradient descent:

<img src="linear_regression_gradient.jpg" width="1600" height="1280" />

The update_weights method below demonstrates one iteration of gradient descent with a loss function for linear regression. Note that one iteration requires a prediction for *each* instance in the training dataset since we are computing m*X[i] + b. We will need to either keep going until some set number of iterations or until there is little improvement in terms of the cost function from the previous iteration.
More on this here: https://stats.stackexchange.com/questions/33136/how-to-define-the-termination-condition-for-gradient-descent
Just know that deep learning libraries run some kind of gradient descent under the covers!

In [20]:
# Below we will use the example of a loss (cost) function for linear regression, which has the equation
# y = m * x + b  where m = a vector of weights
# Mean Squared Error cost function for y = mx + b

# From http://wiki.fast.ai/index.php/Gradient_Descent
def cost_function(x, y, m, b):
  N = len(x)
  total_error = 0.0
  for i in range(N):
    total_error += (y[i] - (m*x[i] + b))**2
  return total_error / N

# From http://wiki.fast.ai/index.php/Gradient_Descent
def update_weights(m, b, X, Y, learning_rate):
  m_deriv = 0
  b_deriv = 0
  N = len(X)
  for i in range(N):
    # Calculate partial derivatives
    # -2x(y - (mx + b))
    m_deriv += -2*X[i] * (Y[i] - (m*X[i] + b))
        
    # -2(y - (mx + b))
    b_deriv += -2*(Y[i] - (m*X[i] + b))

  m += -learning_rate * (m_deriv / float(N)) 
  b += -learning_rate * (b_deriv / float(N))

  return m, b

import pandas as pd
from sklearn import linear_model
from sklearn import datasets

iris = datasets.load_iris()
X = pd.DataFrame(iris.data[:,0:3], columns=iris.feature_names[0:3])
#print X # has three columns: sepal length (cm)  sepal width (cm)  petal length (cm)
#print X.ix[:,0] # just the first column: sepal length
y = pd.DataFrame(iris.data[:,3:4], columns=iris.feature_names[3:4])
#print y.ix[:,0] # one column: petal width

# running one iteration of updating weights on Iris weight.
print update_weights(1, 10, X.ix[:,0], y.ix[:,0], 0.01)

(-0.7148266666666663, 9.707106666666666)


### 6. Regularization


With any loss function, we are presented with an issue. Let's say you found an optimal set of weights that minimize the loss based on the training example data. As explained in [2], "the issue is that this set of W is not necessarily unique: there might be many similar W that correctly classify the examples. One easy way to see this is that if some parameters W correctly classify all examples (so loss is zero for each example), then any multiple of these parameters λW where λ>1 will also give zero loss because this transformation uniformly stretches all score magnitudes and hence also their absolute differences."

Thus, we need to come up with a _regularization penalty_ that is a function of our weights[2]:

<img src="regularization_1.jpg" width="800" height="640" />

which results our loss function being decomposed into _data loss_ and _regularization loss_ [2]:

<img src="regularization_2.jpg" width="800" height="640" />

In the next section, we will see regularization techniques such as dropout used in deep learning.

### 7. References
<pre>
  [1] Fast.ai (http://course.fast.ai/)  
  [2] CS231N (http://cs231n.github.io/)  
  [3] CS224D (http://cs224d.stanford.edu/syllabus.html)  
  [4] Hands on Machine Learning (https://github.com/ageron/handson-ml)  
  [5] Deep learning with Python Notebooks (https://github.com/fchollet/deep-learning-with-python-notebooks)  
  [6] Deep learning by Goodfellow et. al (http://www.deeplearningbook.org/)  
  [7] Neural networks online book (http://neuralnetworksanddeeplearning.com/)  
  [8] https://cs.adelaide.edu.au/~chhshen/teaching/ML_SVR.pdf
  [9] http://www.cs.toronto.edu/~kriz/cifar.html
</pre>

In [21]:
# Solution for L_i is commented out below
# from http://cs231n.github.io/linear-classify/
# def L_i(x, y, W):
#   """
#   unvectorized version. Compute the multiclass svm loss for a single example (x,y)
#   - x is a column vector representing an image (e.g. 3073 x 1 in CIFAR-10)
#     with an appended bias dimension in the 3073-rd position (i.e. bias trick)
#   - y is an integer giving index of correct class (e.g. between 0 and 9 in CIFAR-10)
#   - W is the weight matrix (e.g. 10 x 3073 in CIFAR-10)
#   """
#   delta = 1.0 # see notes about delta later in this section
#   scores = W.dot(x) # scores becomes of size 10 x 1, the scores for each class
#   correct_class_score = scores[y]
#   D = W.shape[0] # number of classes, e.g. 10
#   loss_i = 0.0
#   for j in xrange(D): # iterate over all wrong classes
#     if j == y:
#       # skip for the true class to only loop over incorrect classes
#       continue
#     # accumulate loss for the i-th example
#     loss_i += max(0, scores[j] - correct_class_score + delta)
#   return loss_i